# AI Service Desk Agent — Demo Notebook

Updated demo notebook for the **AI Service Desk Agent** project. This notebook demonstrates:

- Local (offline) demo using provided fallback helpers
- How to create sample data (inventory & tickets)
- Using tools: `inventory_lookup`, `create_ticket`, `get_ticket_status`
- Local agent fallbacks for intake, status, and troubleshooting
- Optional: how to test the remote Gemini-backed FastAPI (`main.py`) if you run it locally


In [ ]:
import sys, os, pathlib
project_root = pathlib.Path('/mnt/data').resolve()
src_path = str(project_root / 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print('Added to sys.path:', src_path)
print('Python version:', sys.version)


## Create sample data (inventory & tickets)
This will write `data/inventory.json` and `data/tickets.json` under `/mnt/data/data`.

In [ ]:
# Create sample data files under /mnt/data/data
import json, pathlib
data_dir = pathlib.Path('/mnt/data/data')
data_dir.mkdir(parents=True, exist_ok=True)

inventory = [
  {"sku":"LAP-001","brand":"BrandA","model":"A123","price":65000,"stock":3, "category":"laptop","description":"BrandA A123 laptop"},
  {"sku":"PRT-001","brand":"PrintCo","model":"PX-100","price":22000,"stock":5, "category":"printer","description":"PrintCo PX-100 printer"}
]
tickets = [
  {"ticket_id":"TICKET-0001","customer_name":"Rita","phone":"+977-9800000000","device":"LAP-001","issue":"won't turn on","status":"received","created_at":"2025-11-01T12:00:00Z","updated_at":"2025-11-01T12:00:00Z","notes":[]}
]

with open(data_dir / 'inventory.json', 'w', encoding='utf-8') as f:
    json.dump(inventory, f, indent=2, ensure_ascii=False)

with open(data_dir / 'tickets.json', 'w', encoding='utf-8') as f:
    json.dump(tickets, f, indent=2, ensure_ascii=False)

print('Wrote sample inventory.json and tickets.json to', str(data_dir))


## Import project modules and local agent fallbacks
Import the tools and the local_* helper functions used for offline testing.

In [ ]:

from tools import inventory_lookup, create_ticket, get_ticket_status
from agents.intake_agent import local_intake_process
from agents.status_agent import local_status_process
from agents.troubleshooting_agent import local_troubleshoot_process

print('Imported modules:')
print(' - inventory_lookup:', inventory_lookup.__name__)
print(' - create_ticket:', create_ticket.__name__)
print(' - get_ticket_status:', get_ticket_status.__name__)
print(' - local_intake_process:', local_intake_process.__name__)
print(' - local_status_process:', local_status_process.__name__)
print(' - local_troubleshoot_process:', local_troubleshoot_process.__name__)


## Inventory lookup demo
Search for a model or brand and show results.

In [ ]:
q = 'BrandA A123'
res = inventory_lookup(q)
print('Query:', q)
print('Result status:', res.get('status'))
print('Result count:', res.get('count'))
for item in res.get('results', []):
    print('-', item['sku'], item['brand'], item['model'], f'Rs.{item.get("price")}', 'stock=', item.get('stock'))


## Create ticket demo (local intake)
Provide a user message and let the local intake processor extract fields and create a ticket.

In [ ]:
# Example user message containing name, phone, and model
user_msg = "Hi, my name is Sita. My phone is +977-9812345678. Model: A123. My laptop won't turn on."
print('User message:', user_msg)
result = local_intake_process(user_msg)
print('\nResult:', result)
# Show ticket saved
from pathlib import Path
import json
tickets_path = Path('/mnt/data/data/tickets.json')
with open(tickets_path, 'r', encoding='utf-8') as f:
    tickets = json.load(f)
print('\nAll tickets:')
for t in tickets:
    print('-', t['ticket_id'], t['customer_name'], t['device'], t['status'])


## Check ticket status demo

In [ ]:
# Use the ticket id from previous step or sample ticket
sample_ticket = 'TICKET-0001'
print('Checking status for', sample_ticket)
status = local_status_process(f'What is the status of ticket {sample_ticket}?')
print(status)


## Troubleshooting demo (local heuristic)
Describe a symptom and get troubleshooting steps.

In [ ]:
symptom = "My printer shows paper jam even when there is no paper."
print('Symptom:', symptom)
ts = local_troubleshoot_process(symptom)
print('\nTroubleshooting result:')
print('Status:', ts.get('status'))
print('Device type:', ts.get('device_type'))
print('Symptoms:', ts.get('symptoms'))
print('Top suggestions:')
for s in ts.get('suggestions', [])[:5]:
    print('-', s)


## Optional: Test remote Gemini-backed backend (if you run `main.py` locally)

If you have started `main.py` (FastAPI) on `http://localhost:8000`, you can test it from this notebook. Kaggle notebooks usually block external API calls — run this only on your local machine or VM where `main.py` is reachable.

In [ ]:
# Optional remote test (only run if main.py is running locally)
import os
try:
    import requests
except Exception:
    requests = None

API_URL = os.getenv('AGENT_API_URL', 'http://localhost:8000').rstrip('/')
if requests is None:
    print('requests library not available in this environment. Install requests to test remote API.')
else:
    try:
        r = requests.get(f'{API_URL}/health', timeout=3)
        print('Health:', r.json())
        payload = {"message": "My laptop won't turn on. Model A123. Name Sita. Phone +9779812345678", "session": {}}
        resp = requests.post(f'{API_URL}/chat', json=payload, timeout=8)
        print('Chat response:', resp.json())
    except Exception as e:
        print('Remote check failed (Is main.py running locally?):', e)
